# Semana 7 — Aprendizado Supervisionado

Notebook com comparação de modelos clássicos de classificação, validação cruzada, ensemble com Gradient Boosting e análise de métricas (accuracy, F1, AUC).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_curve, auc, precision_recall_curve, average_precision_score

plt.style.use('seaborn-v0_8')
np.random.seed(42)


## Carregamento e divisão do dataset

Usamos o dataset de câncer de mama (`sklearn.datasets`), balanceado moderadamente. Separação em treino/teste estratificada para avaliar generalização.

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")
print(pd.Series(y_train).value_counts(normalize=True).rename('prop_train'))


## Modelos comparados e validação cruzada

Avaliamos 4 modelos com hiperparâmetros simples:
- **Regressão Logística** (baseline linear, com padronização).
- **SVM (RBF)** com `C=3`, `gamma='scale'` e probabilidade habilitada.
- **Random Forest** com 200 árvores, profundidade máxima 6.
- **Gradient Boosting** com 120 estimadores, `learning_rate=0.05`.

Usamos validação cruzada estratificada (5 folds) no conjunto de treino para comparar acurácia e F1.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

modelos = {
    'Logistic': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=200))
    ]),
    'SVM_RBF': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', SVC(kernel='rbf', C=3, gamma='scale', probability=True))
    ]),
    'RandomForest': RandomForestClassifier(n_estimators=200, max_depth=6, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=120, learning_rate=0.05, random_state=42)
}

resultados = []
for nome, modelo in modelos.items():
    acc = cross_val_score(modelo, X_train, y_train, cv=cv, scoring='accuracy').mean()
    f1 = cross_val_score(modelo, X_train, y_train, cv=cv, scoring='f1').mean()
    resultados.append({'modelo': nome, 'acc_cv': acc, 'f1_cv': f1})

pd.DataFrame(resultados).sort_values('acc_cv', ascending=False)


## Curvas ROC e PR para os dois melhores modelos

Treinamos em todo o treino e avaliamos no teste, gerando métricas e curvas para comparar calibragem de probabilidade e separação entre classes.

In [ ]:
def avaliar_modelo(nome, modelo):
    modelo.fit(X_train, y_train)
    probas = modelo.predict_proba(X_test)[:, 1]
    preds = modelo.predict(X_test)
    fpr, tpr, _ = roc_curve(y_test, probas)
    prec, rec, _ = precision_recall_curve(y_test, probas)
    return {
        'nome': nome,
        'report': classification_report(y_test, preds, output_dict=True),
        'roc': (fpr, tpr, auc(fpr, tpr)),
        'pr': (prec, rec, average_precision_score(y_test, probas))
    }

melhores = ['GradientBoosting', 'SVM_RBF']
avaliacoes = {n: avaliar_modelo(n, modelos[n]) for n in melhores}

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for nome, dados in avaliacoes.items():
    fpr, tpr, roc_auc = dados['roc']
    axes[0].plot(fpr, tpr, label=f"{nome} (AUC={roc_auc:.3f})")
axes[0].plot([0,1],[0,1], 'k--')
axes[0].set_xlabel('FPR')
axes[0].set_ylabel('TPR')
axes[0].set_title('Curva ROC (teste)')
axes[0].legend()

for nome, dados in avaliacoes.items():
    prec, rec, ap = dados['pr']
    axes[1].plot(rec, prec, label=f"{nome} (AP={ap:.3f})")
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precisão')
axes[1].set_title('Curva Precisão-Recall (teste)')
axes[1].legend()
plt.show()


In [ ]:
# Relatório agregado das métricas de teste
linhas = []
for nome, dados in avaliacoes.items():
    report = dados['report']
    linhas.append({
        'modelo': nome,
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1': report['weighted avg']['f1-score'],
        'roc_auc': dados['roc'][2],
        'ap': dados['pr'][2]
    })

pd.DataFrame(linhas).sort_values('f1', ascending=False)


## Hiperparâmetros testados (resumo)

| Modelo | Principais hiperparâmetros |
| --- | --- |
| Logistic | `C=1.0`, `penalty='l2'`, `max_iter=200`, com `StandardScaler` |
| SVM_RBF | `C=3`, `gamma='scale'`, `probability=True`, com `StandardScaler` |
| RandomForest | `n_estimators=200`, `max_depth=6`, `min_samples_leaf=2`, `random_state=42` |
| GradientBoosting | `n_estimators=120`, `learning_rate=0.05`, `max_depth=3 (padrão)`, `random_state=42` |

Os ensembles (Random Forest e Gradient Boosting) aumentam custo computacional, mas o Gradient Boosting apresentou melhor AUC/F1 nos testes preliminares.